# Задача
---
Вам необходимо предсказать, есть ли в объявлении контактная информация.

Для обучения у вас есть следующие поля:
* `title` - заголовок,
* `description` - описание,
* `subcategory` - подкатегория,
* `category` - категория,
* `price` - цена,
* `region` - регион,
* `city` - город,
* `datetime_submitted` - дата размещения.

Таргет: `is_bad`

Есть два датасета `train.csv` и `val.csv`: `train.csv` содержит больше данных, однако разметка в нём менее точная; в `val.csv` существенно меньше данных, но более точная разметка.
Тестовый датасет, на котором мы оценим решнение, будет больше похож на `val.csv`.
При этом в датасетах могут встречаться (как и, к сожалению, в любых размечаемых данных) некорректные метки.

Данные находятся по ссылке https://drive.google.com/drive/folders/1anZ1bxi5WhPmBlCBnYBYzo4foSgGSee5?usp=sharing. 

Во время запуска вашего решения файл с тестовыми данными будет располагаться по пути `/task-for-hiring-data/test_data.csv`

Файл с результатом работы модели должен представлять из себя `csv` с колонками:
* `index`: `int`, положение записи в файле;
* `prediction`: `float` от 0 до 1, вероятность того, что в объявлении есть контактная информация.

|index  |prediction|
|-------|----------|
|0|0.12|
|1|0.95|
|...|...|
|N|0.68|

После отрабатывания скрипта `run.py` должен записаться csv-файл `/task-for-hiring-data/target_prediction.csv`.

В качестве метрики качества работы вашей модели мы будем использовать усредненный `ROC-AUC` по каждой категории объявлений.

Также есть задача "со звездочкой": предсказать начало и конец контактной информации в описании объявления. Например:
* для строки `Звоните на +7-888-888-88-88, в объявлении некорректный номер`: (11, 26),
* для строки `Звоните на +7-888aaaaa888aaaa88a88, в объявлении некорректный номер`: (11, 33),
* для строки `мой tg: @ivanicki_i на звонки не отвечаю`: (8, 18),
* для строки `мой tg: ivanicki_i на звонки не отвечаю`: (8, 17),
* если в описании объявления (поле `description`) контактов нет, то (None, None)
* если в описании объявления (поле `description`) более одного контакта (`Звоните не 89990000000 или на 89991111111`), то (None, None).

Файл с результатом работы модели должен представлять из себя `csv` с колонками:
* `index`: `int`, положение записи в файле;
* `start`: `int` or `None`, начало маски контакта;
* `end`: `int` or `None`, конец маски контакта.\
(`start` < `end`)

|index  |start|end|
|-------|----------|-----|
|0|None|None|
|1|0|23
|2|31|123
|...|...|
|N|None|None

После отрабатывания скрипта `run.py` должен записаться csv-файл `/task-for-hiring-data/mask_prediction.csv`.

Для задачи со звездочкой метрикой будет усредненный IoU (`Intersection over Union`) по текстам объявлений.

У контейнера не будет доступа в интернет.
Вы можете добавить нужные библиотеки в файл `requirements.txt` или напрямую в `Dockerfile`.

Сделайте форк этого репозитория, а в качестве решения пришлите ссылку на вашу ветку

Удачи :)

# Запуск решения
1. Docker-образ будет собираться командой:\
```docker build -t task_for_hiring -f Dockerfile .```
2. Далее контейнер будет запускаться:\
```docker run -it -v ~/main/task-for-hiring-data:/task-for-hiring-data task_for_hiring python lib/run.py```
3. Файлы с полученным предсказанием должны находиться по тому же пути, что и в тестовом варианте скрипта `lib/run.py`

Обратите внимание, что в контейнере по умолчанию используется python3:

```docker run -it task_for_hiring python -c "import sys; print(sys.version)"```
> 3.7.3 (default, Mar 27 2019, 22:11:17)


In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from scipy.sparse import hstack

In [2]:
train = pd.read_csv('train.csv')
train.shape

(984487, 9)

In [3]:
train.head()

title  \
0                            Диван-кровать   
1  Кожух рулевой колонки Даф хф 91 4509834   
2            Дешёвый буст аккаунтов Dota 4   
3        Телевизор sharp.Смарт тв.Интернет   
4                         Открытка-конверт   

                                         description            subcategory  \
0  Продаем диван-кровать. Удобный механизм - евро...      Мебель и интерьер   
1  Кожух рулевой колонки DAF XF 94 (60066004)/\n ...  Запчасти и аксессуары   
2  ! Буст аккаунтов с ммр выше 1000ммр не беру ! ...      Предложение услуг   
3  Продам телевизор . Диагональ 450.наличие входа...          Аудио и видео   
4  Открытки-конверты ручной работы/\nВыполнены в ...     Коллекционирование   

              category    price                   region          city  \
0      Для дома и дачи   7000.0                   Россия        Москва   
1            Транспорт   2290.0                   Россия        Москва   
2               Услуги    200.0          Северная Осетия   Владикавказ   
3  Бытовая электроника  25000.0  Калининградская область       Советск   
4        Хобби и отдых    150.0      Ставропольский край  Ессентукская   

           datetime_submitted  is_bad  
0  2019-06-01 00:00:15.180656       0  
1  2019-06-01 00:00:44.317933       0  
2  2019-06-01 00:00:50.249692       1  
3  2019-06-01 00:00:50.325799       1  
4  2019-06-01 00:00:56.632655       0

In [4]:
val = pd.read_csv('val.csv')
val.shape

(16237, 9)

In [5]:
val.head()

title  \
0                                      Шины   
1  Продается мобильная перегородка с дверью   
2   Комплект зимних шин на дисках 682/32/64   
3         Кровать-трансормер Дакота сб-4085   
4                    Honda VFR 800 2004 г.в   

                                         description  \
0                                Звонить 89425546881   
1  Мобильная перегородка, предназначена для разгр...   
2  Шины зимние б/у Marshal Assimetric I”Zen KW 61...   
3  Продаю кровать-трансформер производства "Столп...   
4  Мот в отличном состоянии для своих лет, Родной...   

                subcategory         category     price               region  \
0     Запчасти и аксессуары        Транспорт    2000.0     Тульская область   
1  Оборудование для бизнеса      Для бизнеса   10500.0  Вологодская область   
2     Запчасти и аксессуары        Транспорт    4000.0               Россия   
3         Мебель и интерьер  Для дома и дачи   17000.0   Московская область   
4   Мотоциклы и мототехника        Транспорт  235000.0     Брянская область   

       city          datetime_submitted  is_bad  
0  Огаревка  2019-10-10 00:00:25.200714       1  
1   Вологда  2019-10-10 00:03:11.527292       0  
2    Москва  2019-10-10 00:05:07.193248       1  
3     Химки  2019-10-10 00:05:58.165179       0  
4    Брянск  2019-10-10 00:06:19.231151       0

## TF-IDF / One Hot Encoding

In [6]:
train['description'] = train['description'].replace(r'[\W_]+', ' ', regex = True).str.lower()
val['description'] = val['description'].replace(r'[\W_]+', ' ', regex = True).str.lower()

In [7]:
train.head()

title  \
0                            Диван-кровать   
1  Кожух рулевой колонки Даф хф 91 4509834   
2            Дешёвый буст аккаунтов Dota 4   
3        Телевизор sharp.Смарт тв.Интернет   
4                         Открытка-конверт   

                                         description            subcategory  \
0  продаем диван кровать удобный механизм еврокни...      Мебель и интерьер   
1  кожух рулевой колонки daf xf 94 60066004 артик...  Запчасти и аксессуары   
2   буст аккаунтов с ммр выше 1000ммр не беру отв...      Предложение услуг   
3  продам телевизор диагональ 450 наличие входа u...          Аудио и видео   
4  открытки конверты ручной работы выполнены в те...     Коллекционирование   

              category    price                   region          city  \
0      Для дома и дачи   7000.0                   Россия        Москва   
1            Транспорт   2290.0                   Россия        Москва   
2               Услуги    200.0          Северная Осетия   Владикавказ   
3  Бытовая электроника  25000.0  Калининградская область       Советск   
4        Хобби и отдых    150.0      Ставропольский край  Ессентукская   

           datetime_submitted  is_bad  
0  2019-06-01 00:00:15.180656       0  
1  2019-06-01 00:00:44.317933       0  
2  2019-06-01 00:00:50.249692       1  
3  2019-06-01 00:00:50.325799       1  
4  2019-06-01 00:00:56.632655       0

In [8]:
val.head()

title  \
0                                      Шины   
1  Продается мобильная перегородка с дверью   
2   Комплект зимних шин на дисках 682/32/64   
3         Кровать-трансормер Дакота сб-4085   
4                    Honda VFR 800 2004 г.в   

                                         description  \
0                                звонить 89425546881   
1  мобильная перегородка предназначена для разгра...   
2  шины зимние б у marshal assimetric i zen kw 61...   
3  продаю кровать трансформер производства столпл...   
4  мот в отличном состоянии для своих лет родной ...   

                subcategory         category     price               region  \
0     Запчасти и аксессуары        Транспорт    2000.0     Тульская область   
1  Оборудование для бизнеса      Для бизнеса   10500.0  Вологодская область   
2     Запчасти и аксессуары        Транспорт    4000.0               Россия   
3         Мебель и интерьер  Для дома и дачи   17000.0   Московская область   
4   Мотоциклы и мототехника        Транспорт  235000.0     Брянская область   

       city          datetime_submitted  is_bad  
0  Огаревка  2019-10-10 00:00:25.200714       1  
1   Вологда  2019-10-10 00:03:11.527292       0  
2    Москва  2019-10-10 00:05:07.193248       1  
3     Химки  2019-10-10 00:05:58.165179       0  
4    Брянск  2019-10-10 00:06:19.231151       0

In [ ]:
# train[train['is_bad'] == 1]['description'].iloc[7]

In [ ]:
# for i in range(train.shape[1]):
#     print(np.sum(train.iloc[:, i].isnull()))

# for i in range(val.shape[1]):
#     print(np.sum(val.iloc[:, i].isnull()))

In [9]:
# leave only those words that are found in at least 5 objects
vectorizer = TfidfVectorizer(min_df = 5)

# count tf-idf for description
tf_idf_train = vectorizer.fit_transform(train['description'])
tf_idf_val = vectorizer.transform(val['description'])

In [10]:
one_hot = DictVectorizer()
categories = ['subcategory', 'category' , 'region', 'city']

# one hot transformation for text fields
X_train_categ = one_hot.fit_transform(train[categories].to_dict('records'))
X_val_categ = one_hot.transform(val[categories].to_dict('records'))

In [11]:
X_train = hstack([tf_idf_train, X_train_categ])
y_train = train['is_bad']
print('The shape of train data after encoding:', X_train.shape)

The shape of train data after encoding: (984487, 253680)


*Кажется, использовать деревья решений (например, `RandomForest` или `GradientBoosting`)  для такого количества признаков здесь не удасться. Только если попробовать сделать лемматизацию или embedding. Поэтому как выход остается использовать линейные методы.*

In [12]:
X_val = hstack([tf_idf_val, X_val_categ])
y_val = val['is_bad']

## LogisticRegression

*Закомментировал, т.к. поиск лучших парметров занимает достаточно долгое время. Поэтому оставил только результат.*

In [ ]:
# estimator = LogisticRegression()
# param_grid = {'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 10]}

# grid_cv = GridSearchCV(estimator, param_grid, cv = 3)

# grid_cv.fit(X_train, y_train)

In [ ]:
# grid_cv.best_estimator_

### Result: 

```python
LogisticRegression(C=3, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)
```

In [ ]:
# grid_cv.best_score_

In [13]:
lr_clf = LogisticRegression(C=3, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [14]:
lr_clf.fit(X_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=3, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [15]:
accuracy_score(y_train, lr_clf.predict(X_train))

0.9306603337575814

In [16]:
y_pred = lr_clf.predict(X_val)
accuracy = accuracy_score(y_val, y_pred)
print('Accuracy score for val data using LogReg:', accuracy)

Accuracy score for val data using LogReg: 0.890250662068116


In [17]:
y_pred_proba = lr_clf.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print('ROC_AUC score for val data using LogReg:', roc_auc)

ROC_AUC score for val data using LogReg: 0.9343201498731245


# Test 

*Для тестовых данных используем тот классификатор, который показал наилучший результат на val данных.  На данный момент удалось обучить лучше всего только с `LogisticRegression`.*

In [ ]:
best_clf = lr_clf

In [ ]:
# here is your test file (откорректируйте, пожалуйста, расположение файла, если я указал неверно)
test = pd.read_csv('/task-for-hiring-data/test_data.csv')

test['description'] = test['description'].replace(r'[\W_]+', ' ', regex = True).str.lower()
tf_idf_test = vectorizer.transform(test['description'])
X_test_categ = one_hot.transform(test[categories].to_dict('records'))

X_test = hstack([tf_idf_test, X_test_categ])
y_test = test['is_bad']

y_test_pred = best_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print('Accuracy score for test data using best classifier:', accuracy)

y_test_pred_proba = best_clf.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_test_pred_proba)
print('ROC_AUC score for test data using best classifier:', roc_auc)

In [ ]:
target_prediction = pd.DataFrame()
target_prediction['index'] = range(test.shape[0])
target_prediction['prediction'] = y_test_pred_proba

target_prediction.to_csv('/task-for-hiring-data/target_prediction.csv', index=False)
# pd.read_csv('/task-for-hiring-data/target_prediction.csv').head(10)

В случае если не удасться проверить решение на тестовых данных или результат будет ниже порогового $-$ сообщите, пожалуйста.

*Мой телефон:* +79031648555

*telegram:* @mlunov